In [1]:
import pandas as pd
import numpy as np
import requests 
import json
import time
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import math

In [2]:
load_dotenv(override=True)
ID=os.getenv('ID')
TOKEN = os.getenv('TOKEN')



In [3]:
query = "https://api.github.com/users/brotherlogic/events"
query_session = requests.Session()
query_session.auth = (ID,TOKEN)
response = query_session.get(query)
print(response.headers['X-RateLimit-Remaining'])


4998


Importation des librairies et test de la libraire requests

In [4]:
account = pd.read_csv("account_groundTruth[92].csv")
account = account.iloc[:,[1,2]] #used because otherwise the account number are there twice 
# 0              0                       brotherlogic     bot        
#instead of
#0                       brotherlogic     bot
account.describe
print(account.get('login'))
checker = account.values
# print(type(checker))

0        brotherlogic
1           coveralls
2            renovate
3            leocwlam
4           fabric8cd
            ...      
1487         raphaelm
1488       TroyAlford
1489            daffl
1490    bryanyang0528
1491      ConnorRigby
Name: login, Length: 1492, dtype: object


Utilisation de panda pour récupérer la base de données contenant les bots et compte humains.

In [5]:

values = account.values
checker = {}
for array in values:
    checker[array[0]]= array[1]
#Getting dictionnary 

Création d'un dictionnaire pour que lorsque la fonction de recherche pour les bots sera établie, si celle ci a bien détecté un bot ou non.

In [6]:

def checkbot(login):
    return (checker[login] == "bot")

def checkhuman(login):
    return (checker[login] == "human")

checkbot("zouter")
#function used to check whether a account is human or bot
#Probably only need one of the two and not both cant' be a bot if you're human

False

Fonctions servant juste à vérifier correctement dans le dictionnaire

In [7]:
b_set = account.head(25) #bot set of size 25
h_set = account.tail(25) #human set of size 25
m_set = pd.concat([b_set,h_set])                 #mixed set of size 50
m_set = m_set.reset_index(drop = True)           #maybe not needed but "cleans" the dataframe
                                                 # so it goes from 1 to 50 instead of 1 to 25 then 1467 to 1491
print(m_set.to_string())
print(b_set.to_string())
print(h_set.to_string())                        #getting a smaller set to get started

                               login type_GT
0                       brotherlogic     bot
1                          coveralls     bot
2                           renovate     bot
3                           leocwlam     bot
4                          fabric8cd     bot
5                       codecov[bot]     bot
6                         codecov-io     bot
7                     ant-design-bot     bot
8                facebook-github-bot     bot
9                           snyk-bot     bot
10             jnpr-community-netdev     bot
11           dependabot-preview[bot]     bot
12                           ezrobot     bot
13                thelinuxfoundation     bot
14                      Hibernate-CI     bot
15                         cf-gitbot     bot
16                        stale[bot]     bot
17              scrutinizer-notifier     bot
18                         googlebot     bot
19                              bors     bot
20                      k8s-ci-robot     bot
21        

In [8]:
def get_activities(login):
    query_session = requests.Session()
    query_session.auth = (ID,TOKEN)
    i = 1 
    payload  = {'per_page' : '100' ,'page' : '{}'.format(i)}
    response = query_session.get("https://api.github.com/users/{}/events".format(login),params=payload)
    res_json = response.json()
    if (response.status_code != 200):
        with open ("./activities/dead/{}.json".format(login), "w") as outfile:
            json.dump(res_json,outfile,indent= 4)
    else:
        while (i < 3):
            i = i+1
            tmp = (query_session.get("https://api.github.com/users/{}/events".format(login),params=payload)).json()
            res_json += tmp
        with open ("./activities/{}.json".format(login) ,"w") as outfile:
            json.dump(res_json,outfile,indent = 4)
            outfile.close()

#dump json file based on the name of the account


In [10]:
def get_dataframe(login):
    return (pd.read_json("./activities/{}.json".format(login)))

# get_dataframe("ConnorRigby")


In [11]:
def count_type(login):
    df = get_dataframe(login)
    dct = {}
    for val in df['type'].values:
        if val not in dct:
            dct[val]= 1
        else:
            dct[val] = dct[val] +1
    return dct
count_type('brotherlogic')    

{'PushEvent': 66,
 'IssuesEvent': 168,
 'PullRequestEvent': 33,
 'CreateEvent': 33}

Fuction used to get how many different actions the user is doing

In [12]:
def count_repo(login):
    df = get_dataframe(login)
    dct = {}
    # if not df.empty:
    repo = df.get('repo')
    for line in repo:
        tmp = (line['name'])
        if tmp not in dct :
            dct[tmp] = 1
        else : 
            dct[tmp] = dct[tmp] + 1
    return dct            
    # return 
    #Check whether there's not another solution

print(count_repo("brotherlogic"))

{'brotherlogic/adventserver': 138, 'brotherlogic/recordalerting': 132, 'brotherlogic/recordmover': 6, 'brotherlogic/cdprocessor': 21, 'brotherlogic/githubcard': 3}


In [13]:
def varr_time(login):
    df = get_dataframe(login)
    df['created_at'] = df['created_at'].dt.strftime("%s").astype(int) #give time since epoch in seconds
    mean_t = df['created_at'].mean()
    sum = 0
    for i in range (len(df)):
        sum += math.pow((df["created_at"].get(0)- mean_t),2)
    sum = sum/(len(df) -1)
    return sum


def std_dev(login):
    return math.sqrt(varr_time(login))

In [14]:
def gini_repo(login):
    repo_dict = count_repo(login)
    coeff_lst = np.array([ val for val in repo_dict.values()])
    total = 0
    for i, xi in enumerate(coeff_lst[:-1], 1):
        total += np.sum(np.abs(xi - coeff_lst[i:]))
    return total / (len(coeff_lst)**2 * np.mean(coeff_lst))


def gini_type(login):
    repo_dict = count_type(login)
    coeff_lst = np.array([ val for val in repo_dict.values()])
    total = 0
    for i, xi in enumerate(coeff_lst[:-1], 1):
        total += np.sum(np.abs(xi - coeff_lst[i:]))
    return total / (len(coeff_lst)**2 * np.mean(coeff_lst))

gini_type("zbateson")
# gini("ConnorRigby")    
# gini("zserge")

0.3172043010752688

In [16]:

id_set = account.get('login')
for log in id_set:
    print(log)
    get_activities(log)


#GET ALL JSON ACTIVITIES

brotherlogic
coveralls
renovate
leocwlam
fabric8cd
codecov[bot]
codecov-io
ant-design-bot
facebook-github-bot
snyk-bot
jnpr-community-netdev
dependabot-preview[bot]
ezrobot
thelinuxfoundation
Hibernate-CI
cf-gitbot
stale[bot]
scrutinizer-notifier
googlebot
bors
k8s-ci-robot
tessel-bot
resin-io-modules-versionbot[bot]
cfdreddbot
greenkeeper[bot]
manifold-listbot
CLAassistant
gopherbot
sous-machine-account
matzkoh
old-fusion-bot[bot]
aviary2-wf
CFPBot
ININ-TransifexAutomation
stakater-user
pivotal-issuemaster
timsuchanek
ScreensBot
pyup-bot
oca-clabot
kiereleaseuser
fejta-bot
GordonTheTurtle
lock
palashmon
gatsbot[bot]
0crat
rultor
jsf-clabot
dependencies[bot]
devOpsHazelcast
theopenlab-ci
genie-jnosql
ukp-svc-jenkins
miq-bot
travix-frontend-services
ef-carbon-bot
slnode
coreosbot
dsmjs-bot
svc-excavator-bot
hn-bot
Johns-CI-Bot
bors-servo
depfu[bot]
watson-github-bot
openshift-ci-robot
datastax-bot
istio-testing
openshift-bot
hashibot
kubevirt-bot
astropy-bot
HT2Bot
cmsdmwmbot
hubotbot
c

In [17]:
def get_dead(dataframe):
    unused = []
    id_set = dataframe.get('login')
    for log in id_set:
        file = "./activities/dead/{}.json".format(log)
        if os.path.exists(file):
            unused.append(log)
    return unused

dead = get_dead(account)
print(len(account))
new_df = account[~account['login'].isin(dead)]
print(len(new_df))

1492
1489


In [18]:
def get_inactive(df):
    res = []
    id_set = df.get('login')
    for log in id_set:
        tmp = get_dataframe(log)
        if tmp.empty:
            res.append(log)
    return res

In [22]:
dead = get_dead(account)
account = account[~account['login'].isin(dead)]
inactive = get_inactive(account)
account = account[~account['login'].isin(inactive)]

In [21]:
account = account.reset_index(drop=True)
print(account)

                    login type_GT
0            brotherlogic     bot
1               coveralls     bot
2            codecov[bot]     bot
3     facebook-github-bot     bot
4                snyk-bot     bot
...                   ...     ...
1019             raphaelm   human
1020           TroyAlford   human
1021                daffl   human
1022        bryanyang0528   human
1023          ConnorRigby   human

[1024 rows x 2 columns]


In [26]:
index = account[account['type_GT']== 'bot']['login'].count()
bot_account = account.head(index)
index = len(account) - index
human_account = account.tail(index)
bot_account_log = bot_account.get('login')
hum_account_log = human_account.get('login')



              login type_GT
385   mikedidomizio   human
386        yu199195   human
387            edsu   human
388         tkishel   human
389         tschaub   human
...             ...     ...
1019       raphaelm   human
1020     TroyAlford   human
1021          daffl   human
1022  bryanyang0528   human
1023    ConnorRigby   human

[639 rows x 2 columns]


In [27]:
def avg_gini_type(df): #average gini index of the actions done with a dataframe as entry
    counter = 0
    avg = 0
    for login in df:
        # if (login not in dead):
            df = get_dataframe(login)
            # if not df.empty:
            tmp = gini_type(login)
            avg = avg +tmp
            counter = counter + 1
    return avg/counter


In [28]:
bot_avg_gini_type = avg_gini_type(bot_account_log)
hum_avg_gini_type = avg_gini_type(hum_account_log)

In [30]:
print(bot_avg_gini_type)
print(hum_avg_gini_type)

0.23086724181623536
0.36625247252367893


In [31]:
def avg_time_sd(df):  #average standard derivation done with dataframe as entry
    counter = 0
    avg = 0
    for login in df:
        # if (login not in dead):
            df = get_dataframe(login)
            # if not df.empty:
            tmp = std_dev(login)
            avg = avg +tmp
            counter = counter + 1
    return avg/counter

def avg_time_var(df):
    counter = 0
    avg = 0
    for login in df:
        # if (login not in dead):
            df = get_dataframe(login)
            # if not df.empty:
            tmp = varr_time(login)
            avg = avg +tmp
            counter = counter + 1
    return avg/counter


In [32]:
bot_avg_time_var = avg_time_var(bot_account_log)
hum_avg_time_var = avg_time_var(hum_account_log)

bot_avg_time_sd = avg_time_sd(bot_account_log)
hum_avg_time_sd = avg_time_sd(hum_account_log)


In [34]:
print(bot_avg_time_var)
print(hum_avg_time_var)
print(bot_avg_time_sd)
print(hum_avg_time_sd)

3401714838394.3687
5569360245963.754
1206289.7031666667
1861291.9567122082


In [35]:
def array_count_repo(df):
    tab = []
    for login in df:
        # if (login not in dead):
            df = get_dataframe(login)
            # if not df.empty:
            tmp = count_repo(login)
            tab.append(tmp)
    return tab
count_repo_tab = array_count_repo(bot_account.get("login"))

In [36]:
def array_gini_repo(df):
    tab = []
    for login in df:
        # if (login not in dead):
            df = get_dataframe(login)
            # if not df.empty:
            tmp = gini_repo(login)
            tab.append(tmp)
    return tab



In [38]:
bot_gini_repo_array = (array_count_repo(bot_account_log))
hum_gini_repo_array = (array_count_repo(hum_account_log))



In [ ]:
print(bot_gini_repo_array)
print(hum_gini_repo_array)

In [39]:
def array_time_dif(df):
    tab = []
    for login in df:
        if (login not in dead):
            df = get_dataframe(login)
            if not df.empty:
                tmp = varr_time(login)
                tab.append(tmp)
    return tab

In [40]:
bot_array_time_dif = array_time_dif(bot_account_log)
hum_array_time_dif = array_time_dif(hum_account_log)



In [41]:
print(bot_array_time_dif)
print(hum_array_time_dif)

[176091175.42348227, 161835802.7382943, 2099273.0377753456, 3514516544.3320746, 6332004.303992472, 14528465895249.057, 247268677761.04117, 18288675612494.336, 9045910001.116352, 86602809.52263287, 210810948843.32626, 590374329.6962578, 96619191.84285042, 826274352.601411, 19895514932.92097, 7270470359923.411, 12053252960163.084, 6579023047727.704, 6410449686879.492, 253924491646.43613, 12053383583656.5, 10389124.680649748, 132489453953.93306, 2597502205096.9434, 919450421554.138, 206122401481.6832, 14071805872.20242, 9223804790.573109, 911661123298.1595, 11978699170064.537, 36880613431.46038, 2924759346596.1206, 201261750525.8839, 342435670.46089417, 372631073773.29584, 75613716.55211927, 16622290483496.213, 15075069623.16549, 5213578243.163814, 9747295915.35619, 686107432.7795304, 7578637141884.758, 402476496229.11115, 14552271783.407608, 38917709427.1391, 833589599010.6136, 44108549980.1227, 102438056462.49648, 21919352973.95128, 304450568734.712, 15303312090513.611, 78069523835.8107